In [1]:
import nltk
from datetime import datetime
import random as rand
from datetime import date, timedelta
import json
import re
import calendar
from prettytable import PrettyTable
import os
from IPython.display import clear_output
import datetime

import time


In [2]:
# GRAMMAR RULES
grammar = nltk.CFG.fromstring(""" 

m_number -> number m_number | 'one'|'two'|'three'|'four'|'five'|'six'|'seven'|'eight'|'nine'|'ten'|'eleven'|'twelve'|'thirteen'|'fourteen'|'fifteen'|'sixteen'|'seventeen'|'eighteen'|'nineteen'|'twenty'|'thirty'|'forty'|'fifty'|'sixty'|'seventy'|'eighty'|'ninety'|'hundred'

number -> 'one'|'two'|'three'|'four'|'five'|'six'|'seven'|'eight'|'nine'|'ten'|'eleven'|'twelve'|'thirteen'|'fourteen'|'fifteen'|'sixteen'|'seventeen'|'eighteen'|'nineteen'|'twenty'|'thirty'|'forty'|'fifty'|'sixty'|'seventy'|'eighty'|'ninety'|'hundred'

medicine -> 'acetaminophen' | 'ibuprofen' | 'naproxen' | 'amoxicillin' | 'penicillin' | 'azithromycin' | 'fluoxetine' | 'sertraline' | 'duloxetine' | 'venlafaxine' 

dosage_words -> 'once' | 'twice' | 'thrice' | 'times' | 'a' | 'day'

time_interval -> m_number time_interval_words

time_interval_words -> 'hours' | 'hour'


m_dosage -> dosage_words m_dosage | m_number m_dosage | 'once' | 'twice' | 'thrice' | 'times' | 'a' | 'day'


duration -> IN m_number_NNS
m_number_NNS -> m_number NNS | m_number NN



""")



In [3]:
def cykParserV3(input_sentence):
    print("cyk:",input_sentence)
    m_number_rules = grammar.productions()[1:-1]
    ctr=0
    chunk_set = []
    duration = ''
    medicine = ''
    dosage = ''
    time_interval = ''
    matrix = [[set([]) for x  in range(len(input_sentence)-y)] for y in range(len(input_sentence))]
    for x in range(len(input_sentence)): 
        chunk_set.append('0')

    for x in range(0,len(input_sentence)):
        # print("test")        
        for y in range(len(input_sentence)-x):
            if(x==0):
                tag,tokens=input_sentence[y]
                temp = nltk.grammar.Nonterminal(tokens)
                matrix[x][y].add(temp) 

                for prod in grammar.productions():   
                                   
                    if str(prod.lhs()) =='m_number':
                        for rule in prod.rhs():
                            if tag in prod.rhs():
                                matrix[x][y].add(prod.lhs())
                                matrix[x][y].add(nltk.grammar.Nonterminal('number')) 
                  
                    if str(prod.lhs()) =='medicine':
                        for rule in prod.rhs():
                            if tag in prod.rhs():
                                matrix[x][y].add(prod.lhs())                                
                                medicine =  medicine + tag + ' '  

                    if str(prod.lhs()) =='m_dosage':
                        for rule in prod.rhs():
                            if tag in prod.rhs():
                                matrix[x][y].add(prod.lhs())         

                    if str(prod.lhs()) =='dosage_words':
                        for rule in prod.rhs():
                            if tag in prod.rhs():
                                matrix[x][y].add(prod.lhs())    

                    if str(prod.lhs()) =='time_interval_words':
                        for rule in prod.rhs():
                            if tag in prod.rhs():
                                matrix[x][y].add(prod.lhs()) 
                                                     

            if(x>=1):                
                for q1 in range(0,x):

                    for prod in grammar.productions():
                        if len(prod.rhs()) == 2:           
                            A, B = prod.rhs()
                            if prod.rhs()[0] in matrix[q1][y] and prod.rhs()[1] in matrix[x-q1-1][y+q1+1]:
                                matrix[x][y].add(prod.lhs())

                                if(str(prod.lhs())=="duration"):
                                    for ctr in range(x+1):
                                        # if ctr==0:
                                        #     chunk_set[y+ctr] = ['duration',input_sentence[y+ctr][0]]
                                        if ctr!=0:
                                            chunk_set[y+ctr] = ['duration',convert_written_number(input_sentence[y+ctr][0])]

                                if(str(prod.lhs())=="m_dosage"):
                                    for ctr in range(x+1):
                                        chunk_set[y+ctr] = ['dosage',convert_written_number(input_sentence[y+ctr][0])]
                                        # if ctr==0:
                                        #     chunk_set[y+ctr] = ['dosage',convert_written_number(input_sentence[y+ctr][0])]
                                        # if ctr!=0:
                                        #     chunk_set[y+ctr] = ['dosage',convert_written_number(input_sentence[y+ctr][0])]

                                if(str(prod.lhs())=="time_interval"):
                                    for ctr in range(x+1):
                                        chunk_set[y+ctr] = ['time_interval',convert_written_number(input_sentence[y+ctr][0])]
                                        # if ctr==0:
                                        #     chunk_set[y+ctr] = ['time_interval',convert_written_number(input_sentence[y+ctr][0])]
                                        # if ctr!=0:
                                        #     chunk_set[y+ctr] = ['time_interval',convert_written_number(input_sentence[y+ctr][0])]

    for ctr in chunk_set:
        if ctr[0] == 'duration':
            duration = duration + ctr[1] + ' '

        if ctr[0] == 'dosage':
            dosage = dosage + ctr[1] + ' '

        if ctr[0] == 'time_interval':
            time_interval = time_interval + ctr[1] + ' '


    return (medicine,duration,dosage,time_interval)


def convert_written_number(text):
    # Define a regular expression to match written numbers
    pattern = r'\b(zero|one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety|hundred)\b'

    # Use the regular expression to find all matches in the text
    matches = re.findall(pattern, text.lower())

    # Replace each match with its corresponding numerical value
    for match in matches:
        text = re.sub(r'\b{}\b'.format(match), str(convert_to_number(match)), text.lower())

    return text

def convert_to_number(written_number):
    # Define a mapping of written numbers to their numerical values
    mapping = {
        'zero': 0,
        'one': 1,
        'two': 2,
        'three': 3,
        'four': 4,
        'five': 5,
        'six': 6,
        'seven': 7,
        'eight': 8,
        'nine': 9,
        'ten': 10,
        'eleven': 11,
        'twelve': 12,
        'thirteen': 13,
        'fourteen': 14,
        'fifteen': 15,
        'sixteen': 16,
        'seventeen': 17,
        'eighteen': 18,
        'nineteen': 19,
        'twenty': 20,
        'thirty': 30,
        'forty': 40,
        'fifty': 50,
        'sixty': 60,
        'seventy': 70,
        'eighty': 80,
        'ninety': 90,
        'hundred': 100
    }

    # Split the written number into its component words
    words = written_number.split('-')

    # Convert the written number to its numerical form
    if len(words) == 1:
        return mapping[words[0]]
    elif len(words) == 2:
        return mapping[words[0]] + mapping[words[1]]
    elif len(words) == 3:
        return mapping[words[0]] * 100 + mapping[words[2]]
    else:
        raise ValueError("Invalid written number: {}".format(written_number))

   


In [4]:
def sentence_pre_process(sentence):
    tokens = nltk.word_tokenize(sentence)
    tagged_tokens = nltk.pos_tag(tokens)
    temp = cykParserV3(tagged_tokens)
    return temp

def check_days(chunk_sentence):    
    string = str(chunk_sentence).split()
    for char in string:
        if char.isdigit():
            num_temp = char
        if char == 'week' or char == 'weeks':            
            num_temp = int(num_temp) * 7

    return num_temp

In [5]:
def calculate_end_date(days):    
    start_date = date.today()
    end_date = start_date + timedelta(days)
    return(end_date)
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
def sched_date_testing(year_test,month_test,day_test):
    database_today = check_database_today(year_test,month_test,day_test)
    len_temp = len(database_today)
    id_store = []
    meds_strore = []
    intake_today = []
    info_to_render_set = []

    for x in range(len_temp):
        id = database_today[x]['id']
        meds = database_today[x]['medicine']
        temp = check_database_intake(year_test,month_test,day_test,id)[0]
        current_intake = temp['current_intake']
        needed_intake = temp['needed_intake']
        info_to_render = meds + ' ' + '(' + str(current_intake) + '/' + str(needed_intake) +')' 
        info_to_render_set.append(info_to_render)

    temp=[]
    temp2=[]
    temp3=''
    for x in range(8):
        temp=[]
        if x ==0:
            temp.append(day_test)
            for y in range(30):            
                temp.append(' ')
            temp2.append(temp)


        elif x < len(info_to_render_set)+1 and x < 7:        
            temp2.append(str(info_to_render_set[x-1]))

        elif x == 7:
            for y in range(31):            
                temp.append('-')
            temp2.append(temp)

        else:
            for y in range(31):            
                temp.append(' ')
            temp2.append(temp)

    for x in temp2:
        for y in x:
           temp3 = temp3 + str(y)
        temp3 = temp3 + '\n'
    return temp3

# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
def add_sched(duration,date_started,date_ends,medicine,dosage,whole_recorded,interval):

    dosage_num = dosage.split()
    dosage_num = dosage_num[0]

    interval_num = interval.split()
    interval_num = interval_num[0]

    date_ends_s = date_ends - timedelta(days=1)

    date_started_str = date_started.strftime('%Y-%m-%d')
    date_ends_str = date_ends_s.strftime('%Y-%m-%d') 

    random_number = rand.randint(10000, 99999)
    with open("data.json", "r") as f:
        for line in f:
            try:
                data = json.loads(line)
                if data['id'] == random_number:
                    random_number = rand.randint(10000, 99999)               

            except json.JSONDecodeError as e:
                print(f"Error: {e}")

    # new_person = {"id":random_number ,"duration":duration ,"date_started":date_started_str ,"date_ends":date_ends_str ,"medicine":medicine ,"dosage":dosage ,"whole_recorded":whole_recorded }
    new_person = {"id":random_number ,"duration":duration ,"date_started":date_started_str ,"date_ends":date_ends_str ,"medicine":medicine ,"dosage":dosage,'interval':interval ,"whole_recorded":whole_recorded }

    with open("data.json", "a") as f:
        f.write(json.dumps(new_person) + "\n")

    days_difference = date_ends - date_started
    
    for x in range(int(days_difference.days)):
                  
        next_date = date_started + datetime.timedelta(x)
        next_date = next_date.strftime('%Y-%m-%d')
        dosage_day = {"id":random_number ,"current_intake": 0 ,"needed_intake": dosage_num, "date":next_date , 'interval':interval_num,'last_intaked':0,'next_intake':0,}
        with open("data_intake.json", "a") as f:
            f.write(json.dumps(dosage_day) + "\n")
        
 # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------       
def check_database_today(year,month,day): 
    database = []
    if day=='':
        return database

    this_date = datetime.date(int(year),int(month),int(day))   
    
    with open("data.json", "r") as f:
        for line in f:
            
            try:
                data = json.loads(line)   

                start_date = data['date_started']
                s_year, s_month, s_day = start_date.split("-")
                start_date_d = datetime.date(int(s_year),int(s_month),int(s_day))  

                date_ends = data['date_ends']
                e_year, e_month, e_day = date_ends.split("-")
                end_date_d = datetime.date(int(e_year),int(e_month),int(e_day))  

                if start_date_d <= this_date <= end_date_d:
                    database.append(data)

            except json.JSONDecodeError as e:
                print(f"Error: {e}")
    return database

# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
def check_database_intake (year,month,day,id):  
    this_date = datetime.date(int(year),int(month),int(day))

    database = []

    with open("data_intake.json", "r") as f:
        for line in f:
            try:
                data = json.loads(line)
                data_id = data['id']
                today_date = data['date']
                e_year, e_month, e_day = today_date.split("-")
                today_date_d = datetime.date(int(e_year),int(e_month),int(e_day))  

                if today_date_d == this_date and str(id) == str(data_id):
                    database.append(data)

            except json.JSONDecodeError as e:
                print(f"Error: {e}")

    return database

# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

def expand_day(year,month,day):
    global date_today
    temp = check_database_today(year,month,day)
    id_temp = []
    loop_it = True
    
    while loop_it:
        
        print('---------------',month,'-',day,'-',year,'---------------')
        ctr = 0
        for x in temp:      
            
            print(ctr)
            ctr = ctr + 1
            id_temp.append(x['id'])      
            temp2 = check_database_intake (year,month,day,x['id'])      
            print('medicine -> ', x['medicine'])
            print('needed_intake -> ', temp2[0]['current_intake'],'/',temp2[0]['needed_intake']  )
            print('date_started -> ', x['date_started'])
            print('date_ends -> ', x['date_ends'] )
            print('hourly interval -> ',x['interval'] )
            print('last_intaked ->', temp2[0]['last_intaked'])
            print('next_intake ->', temp2[0]['next_intake'])
            transcript_temp = x['whole_recorded']
            print('transcript -> ', transcript_temp[:100] )
            print()
            print()
            print('--------------------------------------------------------------')

        print('1. log an intake')
        print('2. back')
        choice = ''
        choice = input('1. log an intake -- 2. back')
        

        if choice == '1':
            ctr = 0            
            # if int(choice) <= ctr and int(choice)>0:
            this_date = datetime.date(int(year), int(month), int(day))            
            print('what medicine did you take?')
            choice = input('what medicine did you take?')
            this_date = datetime.date(int(year), int(month), int(day))
            
            if this_date == date_today and choice.isdigit():
                choice = int(choice)
                if isinstance(choice, int):                    
                    if len(id_temp) > choice:                        
                        log_intake(year,month,day,id_temp[int(choice)])
                        input('NOT ALLOWED -- press any key to continue')
                        

            else:
                print('you are not allowed to log an intake on dates that are not today',flush=True)
                print('press any key to continue',flush=True)
                input('NOT ALLOWED -- press any key to continue')
                
        elif choice == '2':
            return

        else:
            loop_it = True
            


        clear_output(wait=False)

# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

def render_calendar(year,month):   
    cal = calendar.monthcalendar(year, month)
    table = PrettyTable()
    table.field_names = ["Mo", "Tu", "We", "Th", "Fr", "Sa", "Su"]
    table.title = f"{calendar.month_name[month]} {year}"
    for week in cal:
        if week!=0:
            row = ["" if day == 0 else str(day) for day  in week]

            for x in range(len(row)):
                row[x] = sched_date_testing(year,month,row[x])

        table.add_row(row)
    table.align = "c"
    print(table)


In [6]:
def log_intake(year, month, day, id):
    global remind_me
    this_date = datetime.date(int(year), int(month), int(day))
    date_format = ''
    
    temp = []
    with open("data_intake.json", "r+") as f:
        # id and date
        for line in f:
            try:     
               
                check_date_today = date_today
                
                data = json.loads(line)
                data_id = data['id']
                today_date = data['date']
                e_year, e_month, e_day = today_date.split("-")
                today_date_d = datetime.date(int(e_year), int(e_month), int(e_day))

                temp_hour = time_now.hour + int(data['interval'])
                
                if today_date_d == this_date and str(id) == str(data_id):            
                    if data['next_intake'] == 'EXCEEDED':
                        print('needed dosage for the day failed to be completed (1)',flush=True )
                        print('press any key to continue',flush=True)                        
                        input('press any key to continue -- failed to be completed')
                        # return
                    
# ----------------------------------------------------------------------------------------------------------------------------------------------------
                    if int(data["current_intake"]) != int(data["needed_intake"]):         
                        if data['last_intaked'] != 0 and data['next_intake']!=0 :      
                            # print('remind_me',remind_me)
                            # if remind_me == True:
                            #     date_format == '%H:%M:%S'
                            # else:
                            # date_format == '%H:%M:%S'
                            date_format = '%H:%M:%S.%f'
                            
                            remind_me = False
                            if str(data['next_intake']) != 'EXCEEDED':
                                next_intake_converted = datetime.datetime.strptime(str(data['next_intake']), date_format)
                                next_intake_converted  = next_intake_converted.time()
                                print('Current time -> ',time_now, flush=True)
                                dt = datetime.datetime.combine(datetime.date.today(), time_now)
                                dt_plus_one_hour = dt + datetime.timedelta(hours=int(data['interval']))
                                time_plus_one_hour = dt_plus_one_hour.time()

                                print('Scheduled next intake -> ',temp_hour)

  # ----------------------------------------------------------------------------------------------------------------------------------------------------                      
                                                
                        if data['last_intaked'] == 0 or next_intake_converted<=time_now:                       
                            data["current_intake"] = str(int(data["current_intake"]) + 1)  
                            print('INTAKED', flush=True)                            
                            print('press any key to continue', flush=True)
                            input('INTAKED -- press any key to continue')
                            data['last_intaked'] = time_now.isoformat()

# ----------------------------------------------------------------------------------------------------------------------------------------------------
                            if temp_hour < 24 and int(data["current_intake"]) != int(data["needed_intake"]) :                    
                                next_intake = datetime.time(temp_hour,time_now.minute ,time_now.second,time_now.microsecond)                    
                                data['next_intake'] = next_intake.isoformat()

# ----------------------------------------------------------------------------------------------------------------------------------------------------
                            elif temp_hour >= 24 and int(data["current_intake"]) != int(data["needed_intake"]) :
                                next_intake = 'EXCEEDED'
                                print('needed dosage for the day failed to be completed (2)', flush=True )
                                print('press any key to continue', flush=True)
                                input('press any key to continue -- EXCEEDED')
                                data['next_intake'] = next_intake

# ----------------------------------------------------------------------------------------------------------------------------------------------------
                            elif int(data["current_intake"]) == int(data["needed_intake"]):
                                print('needed dosage for the day completed', flush=True )
                                print('press any key to continue', flush=True)
                                input('press any key to continue -- completed')
                                data['next_intake'] = 0                                          
                                # return
                        else:
                            print('NOT THE RIGHT TIME TO INTAKE THE MEDICINE', flush=True)
                            print('press any key to continue', flush=True)
                            input('press any key to continue -- NOT THE RIGHT TIME') 
                            # return                                                            
                           
                        temp.append(data)                    
                        
                    else:
                        # if data['last_intaked'] == 0 or next_intake_converted<=time_now:  
                        print('needed dosage for the day completed' )
                        temp.append(data)                        
                else:                    
                    temp.append(data)
                                     
            except json.JSONDecodeError as e:
                print(f"Error: {e}")

    with open("data_intake.json", "w") as f:
        f.truncate()

    with open("data_intake.json", "a") as f:
        for x in range(len(temp)):
            f.write(json.dumps(temp[x]) + "\n")


In [7]:
# "`you are sick, you should drink amoxicillin nine times a day for four weeks and every two hours`"
def voice_transcript():  
    sentence = input('recorded voice(simulation only)')

    temp = sentence_pre_process(sentence)
    num_temp = check_days(temp[1])
           

    date_started = date.today()
    date_ends = calculate_end_date(int(num_temp))
    duration = temp[1]
    medicine = temp[0]
    dosage = temp[2]
    interval = temp[3]
    while(duration == '' or interval == '' or date_started == '' or date_ends == '' or medicine == '' or dosage == ''):
        if duration == '' or interval == '' or date_started == '' or date_ends == '' or medicine == '' or dosage == '':
            print('It appears to be that some of the required information are missing...please manually input the missing values')

            if medicine == '':            
                print('what is the medicine called? -> ')
                medicine = input('INPUT A MEDICINE')                            

            if duration == '':            
                print('what is the duration of intaking the medicine? -> ')
                duration = input('INPUT A DURATION')
                temp = sentence_pre_process(duration)
                duration = temp[1]

            if interval == '':            
                print('what is the interval between intakes(in hours)? -> ')
                interval = input('INPUT A INTERVAL')
                temp = sentence_pre_process(interval)
                interval = temp[3]

            if dosage == '':            
                print('what is the dosage instructed?(how many intakes per day) -> ')
                dosage = input('INPUT A DOSAGE')
                temp = sentence_pre_process(dosage)
                dosage = temp[2]


    print('DURATION ->', duration)
    print('INTERVAL ->', interval)
    print('DATE_STARTED -> ', date_started)
    print('DATE_ENDS -> ',date_ends)
    print('MEDICINE ->', medicine)
    print('DOSAGE ->', dosage)
    print('TRANSCRIPT_RECORDED ->', sentence)
    print()
    print()
    print('Adding scheduled intakes successfully')
    print('press any key to continue')
    input('adding schedule successfuly')




    add_sched(duration,date_started,date_ends,medicine,dosage,sentence,interval)


In [8]:
def check_intake_interval(year,month,day):
    temp = check_database_today(year,month,day)
    print(len(temp))
    print()
    for x in temp:
        temp2 = check_database_intake(year,month,day,x['id'])      
        if temp2[0]['last_intaked']==0:
            print('drink your first',x['medicine'],'of the day')



In [9]:

time_now = datetime.time(7, 21, 39, 990662)
date_today = datetime.date.today()
remind_me = False

def main():
    global time_now
    global date_today
    # time_in_seconds = (time_now.hour * 3600) + (time_now.minute * 60) + time_now.second + (time_now.microsecond / 1000000)

    
    today = date_today.strftime('%Y-%m-%d')
    year,month,day = today.split('-')
    year = int(year)
    month = int(month)
    day = int(day)

    if str(day)[0] == '0':
        day = day[1]
    loop_it = True    
    
    while loop_it:
        clear_output(wait=False)
        render_calendar(year,month)    

        print('DATE ->',date_today)
        print('TIME ->',time_now)
        print('--------------------------------')
        
        
        print('1) next month')
        print('2) prev month')
        print('3) select certain date')
        print('4) expand day')
        print('5) enter a scheduled intake')
        print('6) SIMULATE HOUR INCREASE (+)')
        print('7) SIMULATE HOUR DECREASE (-)')
        print('8) SIMULATE DAY INCREASE (+)')
        print('9) SIMULATE DAY DECREASE (-)')
        print('10) SIMULATE REMIND')
        print('0) EXIT')
        choice = input('1) next month -- 2) prev month -- 3) select certain date -- 4) expand day -- 5) enter a schedule')
         

        print('choice',choice)
        if choice == '1':
            month = month + 1
# -------------------------------------------------------------------
        elif choice == '2':
            month = month - 1
# -------------------------------------------------------------------
        elif choice == '3':
            year = int(input('year ->'))
            month = int(input('month ->'))
# -------------------------------------------------------------------
        elif choice == '4':
            # clear_output(wait=False)
            # render_calendar(year,month)  
            print('what day?')
            expand_day_temp = input('what day?')
            clear_output(wait=False)
            expand_day(year,month,expand_day_temp)
# -------------------------------------------------------------------
        elif choice == '5':
            clear_output(wait=False)
            print('ENTER A TRANSCRIPTION')
            voice_transcript()
# -------------------------------------------------------------------
        elif choice == '6':
            print('HOW MANY HOURS TO INCREASE ->')
            hours_to_pass = input('HOW MANY HOURS TO INCREASE: ')
            if hours_to_pass.isdigit():
                
                hours_to_pass = int(hours_to_pass)
                datetime_now = datetime.datetime.combine(datetime.date.today(), time_now)  # convert time to datetime
                datetime_pass = datetime_now + datetime.timedelta(hours=hours_to_pass)  # add timedelta
                time_now = datetime_pass.time()  # extract updated time
            else:
                print()

        elif choice == '7':
            print('HOW MANY HOURS TO DECREASE ->')
            hours_to_pass = input('HOW MANY HOURS TO DECREASE: ')
            if hours_to_pass.isdigit():
                
                hours_to_pass = int(hours_to_pass)
                datetime_now = datetime.datetime.combine(datetime.date.today(), time_now)  # convert time to datetime
                datetime_pass = datetime_now - datetime.timedelta(hours=hours_to_pass)  # add timedelta
                time_now = datetime_pass.time()  # extract updated time
            else:
                print()

        elif choice == '8':
            print('HOW MANY DAYS TO INCREASE ->')
            days_pass = input('HOW MANY DAYS TO INCREASE')
            if days_pass.isdigit():
                date_today = date_today + datetime.timedelta(days=int(days_pass))
            else:
                print()
        
        elif choice == '9':
            print('HOW MANY DAYS TO DECREASE ->')
            days_pass = input('HOW MANY DAYS TO DECREASE')
            if days_pass.isdigit():
                date_today = date_today - datetime.timedelta(days=int(days_pass))
            else:
                print()

        elif choice == '10':
            print('what year?')
            year = input('what year?')
            print('what month?')
            month = input('what month?')
            print('what day?')
            day = input('what day?')
            remind_me(int(year),int(month),int(day))


        elif choice == '0':
            loop_it = False
# ------------------------------------------------------------------- 


            
        else:
            loop_it = True


In [10]:
def remind_me(year,month,day):
    global time_now
    date_format = '%H:%M:%S.%f'
    global remind_me
    remind_me = True
    
    time_now = datetime.time(hour=1, minute=0, second=0, microsecond=0)
    for x in range(23):
        clear_output(wait=False)

        time_now = datetime.time(hour=1, minute=0, second=0, microsecond=5)
        time_now = time_now.replace(hour=time_now.hour+x)
        print(time_now)
        time.sleep(1)
        
        temp = check_database_today(year,month,day)
        
        for x in temp:
            temp2 = check_database_intake (year,month,day,x['id'])  

            if temp2[0]['next_intake'] != 0:
                temp_intake = str(temp2[0]['next_intake'])
                hour, minute, second =temp_intake.split(':')
                next_intake_converted = datetime.time(hour=int(hour), minute=int(minute), second=0 , microsecond=5)
            
            if temp2[0]['next_intake'] == 0 or next_intake_converted <= time_now:
                print(temp2)
                print('you have a scheduled intake of medicine at this hour...')
                print('do you want to log it?')
                print('1) yes')
                print('2) no')
                print('3) end simulation')
                
                choice = str(input())

                if choice == '1':
                    log_intake(year, month, day, x['id'])
                elif choice == '2':
                    print('SKIPPED')
                elif choice == '3':
                    main()
                else:
                    print()

    remind_me = False
        # time.sleep(1)
        

 


In [11]:
def expand_day_remind(year,month,day,time):
    global date_today
    temp = check_database_today(year,month,day)
    id_temp = []
    loop_it = True
    
    while loop_it:
        
       
        ctr = 0
        for x in temp:      
            
            print(ctr)
            ctr = ctr + 1
            id_temp.append(x['id'])      
            temp2 = check_database_intake (year,month,day,x['id'])      

            if temp2[0]['next_intake'] == time:
                print('--------------------------------------------------------------')
                print('medicine -> ', x['medicine'])
                print('needed_intake -> ', temp2[0]['current_intake'],'/',temp2[0]['needed_intake']  )
                print('date_started -> ', x['date_started'])
                print('date_ends -> ', x['date_ends'] )
                print('hourly interval -> ',x['interval'] )
                print('last_intaked ->', temp2[0]['last_intaked'])
                print('next_intake ->', temp2[0]['next_intake'])
                transcript_temp = x['whole_recorded']
                print('transcript -> ', transcript_temp[:100] )
                print()
                print()
                print('--------------------------------------------------------------')


                print('1. log an intake')
                print('2. back')
                choice = ''
                choice = input('1. log an intake -- 2. back')
                
                if choice == '1':
                    ctr = 0        
                    this_date = datetime.date(int(year), int(month), int(day))            
                    print('what medicine did you take?')
                    choice = input('what medicine did you take?')
                    this_date = datetime.date(int(year), int(month), int(day))
                    
                    if this_date == date_today and choice.isdigit():
                        choice = int(choice)
                        if isinstance(choice, int):                    
                            if len(id_temp) > choice:                        
                                log_intake(year,month,day,id_temp[int(choice)])
                                input('NOT ALLOWED -- press any key to continue')
                        

            # else:
            #     print('you are not allowed to log an intake on dates that are not today',flush=True)
            #     print('press any key to continue',flush=True)
            #     input('NOT ALLOWED -- press any key to continue')
                
                elif choice == '2':
                    return

                else:
                    loop_it = True
            


                clear_output(wait=False)

In [12]:
main()


+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                      May 2023                                                                                                                      |
+----------------------------------+----------------------------------+----------------------------------+----------------------------------+----------------------------------+----------------------------------+----------------------------------+
|                Mo                |                Tu                |                We                |                Th                |                Fr                |                Sa                |                Su                |
+-----------